In [1]:
###############################################################################
# Import packages
###############################################################################
import time
import pandas as pd
import numpy as np
from scipy.stats import skew
import itertools

# Load functions
from bin_packing_functions import generate_bin_packing_instance, solve_bin_packing_with_size_and_weight, fun_all_sums
from bin_packing_functions import fun_shapley_value, fun_convert_time, fun_save_file

In [5]:
###############################################################################
# Generate random instances - settings
###############################################################################
num_items_list = [10]#, 6, 7, 8, 9, 10, 11, 12]#13, 14, 15] # All possible instance sizes (number of customers for an instance)
start_value = 1 # <- Der Wert hier wird als Instanz_id reingeschrieben, hab das drin weil ich immer mehrere Skripte Parallel laufen lasse, da können nicht alle bei 0 starten
number_of_instances_per_size = 1 # Determines the number of instances that are generated for each instance size (number of customers)
prints = True # Show prints and interim results during generating
print_details = False

# Create DataFrame to merge final instances
df_final = pd.DataFrame()
run_time = {}

###############################################################################
# START GENERAITING
###############################################################################
for num_items in num_items_list:
    instance_id = 100000 * num_items * start_value # -> Random gesetzt, hauptsache es gibt bei Parallelisierung keine Überschneidung der Indizes
    start = time.time() # Start time count
    if (number_of_instances_per_size > 1): prints, print_details = False, False # No prints for more than one instance

    for inumber in range(0, number_of_instances_per_size):
        ###############################################################################
        # Generate a random instance
        ###############################################################################

        instance_df = generate_bin_packing_instance(num_items=num_items)

        # Add instance id as to each row as first column and update instance id for next iteration
        instance_df.insert(0, 'Instance ID', instance_id)
        instance_id += 1

        # Add number of customers as second feature
        instance_df.insert(1, 'Number Items', num_items)

        if (prints == True): print('############### INSTANCE ###############'); display(instance_df)

        ###############################################################################
        # Add features bin utilizations and combinabilities
        ###############################################################################

        # Extract weight and size values of the items as well as the bin weight and size capacities
        weights = instance_df['Item Weight']
        sizes = instance_df['Item Size']
        bin_weight = instance_df.loc[0, 'Bin Weight']
        bin_size = instance_df.loc[0, 'Bin Size']

        # Compute sum, product and quotient of weight and size for all items
        instance_df['Weight Size Sum'] = weights + sizes
        instance_df['Item Volume'] = weights * sizes
        instance_df['Item Density'] = weights / sizes

        # Compute the single bin utilization for all items in terms of weight, size and total bin utilization
        instance_df['Bin Utilization Weight'] = weights / bin_weight
        instance_df['Bin Utilization Size'] = sizes / bin_size
        instance_df['Total Bin Utilization'] = (weights + sizes) / (bin_weight + bin_size)

        # Compute the number of possible item combinations to put in a bin for all items in terms of weight, size and total bin utilization
        num_combinations_list = {'Weight': [], 'Size': [], 'Total': []}
        num_perfect_combinations_list = {'Weight': [], 'Size': [], 'Total': []}

        for index, values in enumerate(zip(weights, sizes)):
            # Weight combinability
            weight = values[0]
            remaining_bin_weight = bin_weight - weight
            remaining_item_weights = [weights[i] for i in list(weights.index) if i is not index]
            all_sums_weights = fun_all_sums(remaining_item_weights)
            num_combinations_weight = np.sum([i <= remaining_bin_weight for i in all_sums_weights])
            num_perfect_combinations_weight = np.sum([i == remaining_bin_weight for i in all_sums_weights])
            num_combinations_list['Weight'] += [num_combinations_weight]
            num_perfect_combinations_list['Weight'] += [num_perfect_combinations_weight]

            # Size combinability
            size = values[1]
            remaining_bin_size = bin_size - size
            remaining_item_sizes = [sizes[i] for i in list(sizes.index) if i is not index]
            all_sums_sizes = fun_all_sums(remaining_item_sizes)
            num_combinations_size = np.sum([i <= remaining_bin_size for i in all_sums_sizes])
            num_perfect_combinations_size = np.sum([i == remaining_bin_size for i in all_sums_sizes])
            num_combinations_list['Size'] += [num_combinations_size]
            num_perfect_combinations_list['Size'] += [num_perfect_combinations_size]

            # Total combinability
            num_combinations_total = np.sum([(weight <= remaining_bin_weight) & (size <= remaining_bin_size) 
                                    for weight, size in zip(all_sums_weights, all_sums_sizes)])
            num_perfect_combinations_total = np.sum([(weight == remaining_bin_weight) & (size == remaining_bin_size) 
                                    for weight, size in zip(all_sums_weights, all_sums_sizes)])
            num_combinations_list['Total'] += [num_combinations_total]
            num_perfect_combinations_list['Total'] += [num_perfect_combinations_total] 

            if (print_details == True):
                if (index == 0): print('############### COMBINABILITY FEATURES ###############')
                print('\n- Item {}:\n  Remaining weight: {} - {} = {}\n  Remaining item weights: {}'.format(index+1, bin_weight, weight, remaining_bin_weight, remaining_item_weights))
                print('  Number of possible bin combinations (weight): {}'.format(num_combinations_weight))
                print('  Number of bin combinations for perfect utilization (weigth): {}'.format(num_perfect_combinations_weight))
                print('  Remaining size: {} - {} = {}\n  Remaining item sizes: {}'.format(bin_size, size, remaining_bin_size, remaining_item_sizes))
                print('  Number of possible bin combinations (size): {}'.format(num_combinations_size))
                print('  Number of bin combinations for perfect utilization (size): {}'.format(num_perfect_combinations_size))
                print('=> Number of possible total bin combinations:', num_combinations_total)
                print('=> Number of total bin combinations for perfect utilization:', num_perfect_combinations_total)

        # Add results as columns to the DataFrame
        instance_df.loc[:, 'Weight Bin Combinations'] = num_combinations_list['Weight']
        instance_df.loc[:, 'Size Bin Combinations'] = num_combinations_list['Size']
        instance_df.loc[:, 'Total Bin Combinations'] = num_combinations_list['Total']
        instance_df.loc[:, 'Perfect Weight Bin Combinations'] = num_perfect_combinations_list['Weight']
        instance_df.loc[:, 'Perfect Size Bin Combinations'] = num_perfect_combinations_list['Size']
        instance_df.loc[:, 'Perfect Total Bin Combinations'] = num_perfect_combinations_list['Total']

        if (prints == True): display(instance_df)

        ###############################################################################
        # Add statistics of TSP/CVRP as features
        ###############################################################################

        # Extract weight and size values of the items
        weights = instance_df['Item Weight']
        sizes = instance_df['Item Size']

        # Compute further statistics for weights and sizes of the items and add each of them as a column
        for statistic in [np.sum, np.mean, np.std, np.max, np.min]:
            instance_df['Weight ' + statistic.__name__.capitalize()] = statistic(weights)
            instance_df['Size ' + statistic.__name__.capitalize()] = statistic(sizes)

        # Calculate the correlation between weights and sizes of the items as well as the skewness for weights and sizes and add statistics to each row as a column
        instance_df['Correlation'] = np.corrcoef(weights, sizes)[0, 1] # Select first row and second column of correlation matrix
        instance_df['Skewness Weight'] = skew(weights)
        instance_df['Skewness Size'] = skew(sizes)

        # Define columns for displaying the instance
        instance_features = ['Instance ID', 'Number Items', 'Item ID', 'Item Weight', 'Item Size', 'Bin Weight', 'Bin Size']
        statistical_features = ['Weight Sum', 'Size Sum', 'Weight Mean', 'Size Mean', 'Weight Std', 'Size Std', 'Weight Max', 
                                'Size Max', 'Weight Min', 'Size Min', 'Correlation', 'Skewness Weight', 'Skewness Size']
        
        if (prints == True): print('\n############### STATISTICAL FEATURES ###############'); display(instance_df[instance_features + statistical_features])

        ###############################################################################
        # Solve instance
        ###############################################################################

        total_bins = solve_bin_packing_with_size_and_weight(instance_df)
        if (prints == True): print('\n############### SOLVE INSTANCE ###############'); print('Total bins:', total_bins)

        ###############################################################################
        # Calculate total costs for all subsets
        ###############################################################################

        # Determine all subsets of all items
        all_items = range(1, num_items+1)
        list_of_all_subsets = []

        for item in all_items:
            subsets = itertools.combinations(all_items, item)
            for subset in subsets:
                list_of_all_subsets.append(subset)

        # Check if the correct number of subsets was created (number of possible subsets: 2^n -1)
        if (prints == True): print('Correct number of subsets:', len(list_of_all_subsets) == 2**num_items - 1)

        # Initialize dictionary with subset total cost / total number of bins
        subset_total_costs = {}

        # Evaluate total cost / total number of bins for each subset and store in dictionary
        for subset in list_of_all_subsets:
            # Select rows of the items contained in the subset and solve the subset instance
            subset_instance = instance_df[instance_df['Item ID'].isin(subset)]
            sub_total_bins = solve_bin_packing_with_size_and_weight(subset_instance)

            # Add the total costs to the dictionary
            subset_total_costs[subset] = sub_total_bins

        ###############################################################################
        # Calculate marginal costs (solve instance without item)
        ###############################################################################

        # Initialize an empty list to store the marginal costs
        all_marginal_costs = []

        # Iterate over all customers to get the costs of the subset without the customer and compute the marginal costs
        for i in all_items:
            set_without_customer = set(all_items) - set([i])
            bins_without_customer = subset_total_costs[tuple(set_without_customer)]
            marginal_costs = total_bins - bins_without_customer

            if (print_details == True):
                if (i == 1): print('\n############### FEATURE MARGINAL COSTS ###############')
                print('  - Item {}\n      Set without item: {}\n      Costs without item: {}'.format(i, set_without_customer, bins_without_customer))
                print('      Marginal costs/bins: {} - {} = {}'.format(total_bins, bins_without_customer, marginal_costs))

            # Append the marginal cost to the list
            all_marginal_costs.append(marginal_costs)

        # Add the marginal costs to the DataFrame and also a column for total bins
        instance_df['Marginal Costs/Bins'] = all_marginal_costs
        instance_df['Total Bins'] = total_bins

        ###############################################################################
        # Calculate Shapley values
        ###############################################################################

        # Calculate Shapley value for each item
        shapley_values = []
        for i in range(1, num_items+1):
            shapley_value = fun_shapley_value(player_index=i, characteristic_function=subset_total_costs, prints=print_details)
            shapley_values.append(shapley_value)

        # Append Shapley values as column
        instance_df['Shapley Value'] = shapley_values

        # Define columns for displaying the instance
        cost_features = ['Marginal Costs/Bins', 'Total Bins', 'Shapley Value']

        if (prints == True): display(instance_df[instance_features + cost_features])

        ###############################################################################
        # Add ratio features
        ###############################################################################

        # Define features for which the ratio should be computed
        no_ratio_features = ['Instance ID', 'Number Items', 'Item ID', 'Bin Weight', 'Bin Size', 'Total Bins', 'Shapley Value'] + statistical_features
        ratio_features = [i for i in instance_df.columns if i not in no_ratio_features]

        # Compute mean of all ratio_features (without the depot) and divide their values by mean to obtain ratio column which is inserted before the Shapley value
        for feature in ratio_features:
            mean = np.mean(instance_df[feature])
            if (mean != 0): ratio = instance_df[feature] / mean # When ther is no perfect bin combination for all items the mean would be 0 -> ratio = np.nan
            else: ratio = 0
            instance_df.insert(loc=len(instance_df.columns) - 1, column=feature + str(' Ratio'), value=ratio)
        
        if (prints == True):
            print('\n############### RATIO FEATURES ###############')
            print('No ratio features:\n', no_ratio_features)
            display(instance_df[[i + str(' Ratio') for i in ratio_features]])

        # Merge instances
        df_final = pd.concat([df_final, instance_df], ignore_index=True)

    # Stop time count for instance size and make print to show progress during generation
    run_time[num_items] = time.time() - start
    if (num_items_list[-1] > 10): print('- Instance size {} completed! Run time: {}'.format(num_items, fun_convert_time(seconds=run_time[num_items])))

    # View final instance when there was only one generated per instance size
    if (number_of_instances_per_size == 1): print('\n############### FINAL INSTANCE ###############'); display(instance_df)

    # Else view run times and final DataFrame with merged instances
    elif (instance_id == (100000 * num_items_list[-1] * start_value) + number_of_instances_per_size):
        print('-> Total run time: ', fun_convert_time(seconds=sum(run_time.values())))
        display({'Instance size: {} run time'.format(key): fun_convert_time(seconds=value) for key, value in run_time.items()})
        display(df_final)

# Save file
fun_save_file(data=df_final, subfolder_path='..\\..\\01_data\\03_bin_packing', name='bin_packing_instances.xlsx')

############### INSTANCE ###############


,Instance ID,Number Items,Item ID,Item Weight,Item Size,Bin Weight,Bin Size
0,1000000,10,1,5,4,10,10
1,1000000,10,2,4,1,10,10
2,1000000,10,3,3,4,10,10
3,1000000,10,4,2,3,10,10
4,1000000,10,5,1,4,10,10
5,1000000,10,6,3,4,10,10
6,1000000,10,7,4,5,10,10
7,1000000,10,8,3,6,10,10
8,1000000,10,9,1,1,10,10
9,1000000,10,10,5,4,10,10


,Instance ID,Number Items,Item ID,Item Weight,Item Size,Bin Weight,Bin Size,Weight Size Sum,Item Volume,Item Density,Bin Utilization Weight,Bin Utilization Size,Total Bin Utilization,Weight Bin Combinations,Size Bin Combinations,Total Bin Combinations,Perfect Weight Bin Combinations,Perfect Size Bin Combinations,Perfect Total Bin Combinations
0,1000000,10,1,5,4,10,10,9,20,1.250000,0.5,0.4,0.45,29,27,16,11,7,1
1,1000000,10,2,4,1,10,10,5,4,4.000000,0.4,0.1,0.25,42,55,33,15,17,0
2,1000000,10,3,3,4,10,10,7,12,0.750000,0.3,0.4,0.35,54,27,21,16,7,0
3,1000000,10,4,2,3,10,10,5,6,0.666667,0.2,0.3,0.25,68,30,25,22,3,0
4,1000000,10,5,1,4,10,10,5,4,0.250000,0.1,0.4,0.25,80,27,25,21,7,0
5,1000000,10,6,3,4,10,10,7,12,0.750000,0.3,0.4,0.35,54,27,21,16,7,0
6,1000000,10,7,4,5,10,10,9,20,0.800000,0.4,0.5,0.45,42,22,17,15,11,2
7,1000000,10,8,3,6,10,10,9,18,0.500000,0.3,0.6,0.45,54,11,11,16,7,0
8,1000000,10,9,1,1,10,10,2,1,1.000000,0.1,0.1,0.10,80,55,43,21,17,2
9,1000000,10,10,5,4,10,10,9,20,1.250000,0.5,0.4,0.45,29,27,16,11,7,1



############### STATISTICAL FEATURES ###############


,Instance ID,Number Items,Item ID,Item Weight,Item Size,Bin Weight,Bin Size,Weight Sum,Size Sum,Weight Mean,Size Mean,Weight Std,Size Std,Weight Max,Size Max,Weight Min,Size Min,Correlation,Skewness Weight,Skewness Size
0,1000000,10,1,5,4,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
1,1000000,10,2,4,1,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
2,1000000,10,3,3,4,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
3,1000000,10,4,2,3,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
4,1000000,10,5,1,4,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
5,1000000,10,6,3,4,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
6,1000000,10,7,4,5,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
7,1000000,10,8,3,6,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
8,1000000,10,9,1,1,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226
9,1000000,10,10,5,4,10,10,31,36,3.1,3.6,1.374773,1.496663,5,6,1,1,0.262445,-0.180116,-0.551226


...


############### RATIO FEATURES ###############
No ratio features:
 ['Instance ID', 'Number Items', 'Item ID', 'Bin Weight', 'Bin Size', 'Total Bins', 'Shapley Value', 'Weight Sum', 'Size Sum', 'Weight Mean', 'Size Mean', 'Weight Std', 'Size Std', 'Weight Max', 'Size Max', 'Weight Min', 'Size Min', 'Correlation', 'Skewness Weight', 'Skewness Size']


,Item Weight Ratio,Item Size Ratio,Weight Size Sum Ratio,Item Volume Ratio,Item Density Ratio,Bin Utilization Weight Ratio,Bin Utilization Size Ratio,Total Bin Utilization Ratio,Weight Bin Combinations Ratio,Size Bin Combinations Ratio,Total Bin Combinations Ratio,Perfect Weight Bin Combinations Ratio,Perfect Size Bin Combinations Ratio,Perfect Total Bin Combinations Ratio,Marginal Costs/Bins Ratio
0,1.612903,1.111111,1.343284,1.709402,1.114413,1.612903,1.111111,1.343284,0.545113,0.876623,0.701754,0.670732,0.777778,1.666667,0
1,1.290323,0.277778,0.746269,0.341880,3.566122,1.290323,0.277778,0.746269,0.789474,1.785714,1.447368,0.914634,1.888889,0.000000,0
2,0.967742,1.111111,1.044776,1.025641,0.668648,0.967742,1.111111,1.044776,1.015038,0.876623,0.921053,0.975610,0.777778,0.000000,0
3,0.645161,0.833333,0.746269,0.512821,0.594354,0.645161,0.833333,0.746269,1.278195,0.974026,1.096491,1.341463,0.333333,0.000000,0
4,0.322581,1.111111,0.746269,0.341880,0.222883,0.322581,1.111111,0.746269,1.503759,0.876623,1.096491,1.280488,0.777778,0.000000,0
5,0.967742,1.111111,1.044776,1.025641,0.668648,0.967742,1.111111,1.044776,1.015038,0.876623,0.921053,0.975610,0.777778,0.000000,0
6,1.290323,1.388889,1.343284,1.709402,0.713224,1.290323,1.388889,1.343284,0.789474,0.714286,0.745614,0.914634,1.222222,3.333333,0
7,0.967742,1.666667,1.343284,1.538462,0.445765,0.967742,1.666667,1.343284,1.015038,0.357143,0.482456,0.975610,0.777778,0.000000,0
8,0.322581,0.277778,0.298507,0.085470,0.891530,0.322581,0.277778,0.298507,1.503759,1.785714,1.885965,1.280488,1.888889,3.333333,0
9,1.612903,1.111111,1.343284,1.709402,1.114413,1.612903,1.111111,1.343284,0.545113,0.876623,0.701754,0.670732,0.777778,1.666667,0



############### FINAL INSTANCE ###############


,Instance ID,Number Items,Item ID,Item Weight,Item Size,Bin Weight,Bin Size,Weight Size Sum,Item Volume,Item Density,...,Bin Utilization Size Ratio,Total Bin Utilization Ratio,Weight Bin Combinations Ratio,Size Bin Combinations Ratio,Total Bin Combinations Ratio,Perfect Weight Bin Combinations Ratio,Perfect Size Bin Combinations Ratio,Perfect Total Bin Combinations Ratio,Marginal Costs/Bins Ratio,Shapley Value
0,1000000,10,1,5,4,10,10,9,20,1.250000,...,1.111111,1.343284,0.545113,0.876623,0.701754,0.670732,0.777778,1.666667,0,0.476190
1,1000000,10,2,4,1,10,10,5,4,4.000000,...,0.277778,0.746269,0.789474,1.785714,1.447368,0.914634,1.888889,0.000000,0,0.187302
2,1000000,10,3,3,4,10,10,7,12,0.750000,...,1.111111,1.044776,1.015038,0.876623,0.921053,0.975610,0.777778,0.000000,0,0.449206
3,1000000,10,4,2,3,10,10,5,6,0.666667,...,0.833333,0.746269,1.278195,0.974026,1.096491,1.341463,0.333333,0.000000,0,0.413492
4,1000000,10,5,1,4,10,10,5,4,0.250000,...,1.111111,0.746269,1.503759,0.876623,1.096491,1.280488,0.777778,0.000000,0,0.419444
5,1000000,10,6,3,4,10,10,7,12,0.750000,...,1.111111,1.044776,1.015038,0.876623,0.921053,0.975610,0.777778,0.000000,0,0.449206
6,1000000,10,7,4,5,10,10,9,20,0.800000,...,1.388889,1.343284,0.789474,0.714286,0.745614,0.914634,1.222222,3.333333,0,0.483730
7,1000000,10,8,3,6,10,10,9,18,0.500000,...,1.666667,1.343284,1.015038,0.357143,0.482456,0.975610,0.777778,0.000000,0,0.513492
8,1000000,10,9,1,1,10,10,2,1,1.000000,...,0.277778,0.298507,1.503759,1.785714,1.885965,1.280488,1.888889,3.333333,0,0.131746
9,1000000,10,10,5,4,10,10,9,20,1.250000,...,1.111111,1.343284,0.545113,0.876623,0.701754,0.670732,0.777778,1.666667,0,0.476190


File saved successfully!
